In [2]:
!pip install gensim
import os
import re
import string
import operator
import warnings
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.externals import joblib

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.utils import lemmatize
from gensim.models import Phrases 
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.models import TfidfModel
from gensim.test.utils import datapath
from pprint import pprint

nltk.download('stopwords')
nltk.download('wordnet')

import gc

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christian.conroy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/christian.conroy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stops = stopwords.words('english')
stops.extend(['cik', 'company', 'name', 'clancy', 'corpform','type','kfiling', 'date', 'accession', 'number', 'item', 'table', 'contents'])
st1= ['after', 'afterwards','again','against', 'all', 'almost','alone','along','already','also','although', 'always', 
      'am','among','amongst', 'amoungst', 'amount','an','and','another','any','anyhow','anyone','anything', 'anyway',
      'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 
      'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 
      'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant','co','con','could','couldnt','cry','de',
      'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else',
      'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except',
      'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found',
      'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 
      'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how',
      'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed', 'interest', 'into', 'is', 'it', 'its', 'itself',
      'keep', 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'me', 'meanwhile',
      'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 
      'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 
      'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 
      'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 
      'put', 'rather', 're', 'same', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 
      'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 
      'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'their', 
      'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon',
      'these', 'they', 'thick', 'thin', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru',
      'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 
      'up', 'upon', 'us', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when',  'whence', 
      'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which',
      'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'without',
      'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves']
# More extension 
stops.extend(st1)

In [3]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(token)
    return result

def build_text(doc_set):
    for line in doc_set:
        yield gensim.utils.simple_preprocess(line, deacc=True, min_len=3)
        
def process_texts(texts):
    """
    Function to process texts. Following are the steps we take:
    
    1. Stopword Removal.
    2. Collocation detection.
    3. Lemmatization (not stem since stemming can reduce the interpretability).
    
    Parameters:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    lemmatizer = WordNetLemmatizer()
    texts = [[lemmatizer.lemmatize(word) for word in line] for line in texts]
    #texts = [[word.split('/')[0] for word in lemmatize(' '.join(line), allowed_tags=re.compile('(NN)'), min_length=3)] for line in texts]
    return texts

# So we're setting the word limit in one go here.
# I skipped the lemmative part as I just couldn't get it working with the bytes strings issue

def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    numtopics = []
    for num_topics in range(1, limit):
        numtopics.append(num_topics)
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, per_word_topics = True, random_state=100)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
    
    return lm_list, c_v, numtopics


## 10ks

Option 1: https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/scikit_learn_iris.py

In [ ]:
import os 
import re
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
import s3fs
import time

if __name__ == '__main__':
    #### Parse Arguments
    parser = argparse.ArgumentParser()
    
    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    args = parser.parse_args()
    
    ###### All operations start here
    start_time = time.time()
    #### Bring in the data
    data_list = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    comp10ks = []
    for i in range(0,len(data_list)):
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    else: 
        try:
        s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
        text_file = s[0].str.cat(sep=' ')
        comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
        except:
            pass  
    # Convert All to Pandas Dataframe 
    cols = ['year', 'compname', "text"]
    comp10ks = pd.DataFrame(comp10ks, columns=cols)
    comp10ks.Year = pd.to_datetime(comp10ks.year).dt.to_period('y')
    #### Pre-process and clean
    comp10ks['text'] = comp10ks['text'].map(lambda x: re.sub('[,\.!?]', '', x))
    comp10ks['text'] = comp10ks.text.str.replace(r'<.*?>', '', regex= True)
    comp10ks['text'] = comp10ks.text.str.replace(r'&#\d+;', '', regex= True)
    # Convert the titles to lowercase
    comp10ks['text'] = comp10ks['text'].map(lambda x: x.lower())
    comp10ks['text'] = comp10ks.text.str.replace('\n', '')
    comp10ks['text'] = comp10ks.text.str.replace('\t', '')
    comp10ks['text'] = comp10ks.text.str.replace('&nbsp;','')
    # Pull out fields
    comp10ks['bus_address'] = comp10ks.text.str.extract('business address: street 1:(.*?)mail address')
    comp10ks['mail_address'] = comp10ks.text.str.extract('mail address: street 1:(.*?)10-k')
    # Scrape out risk text using url
    comp10ks['riskfac'] = comp10ks.text.str.extract('item 1a(.*?)item 2')
    comp10ks['mandisc'] = comp10ks.text.str.extract('item 7(.*?)item 8')
    comp10ks['fullrisktext'] = comp10ks['riskfac'] + comp10ks['mandisc']
    # Pull out stopwords
    comp10ks['text'] = comp10ks['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stops]))
    comp10ks['fullrisktext'] = comp10ks['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stops]))
    # Tokenize
    comp10ks['words'] = comp10ks['text'].map(preprocess)
    # Create Bag of Words and convert to Pandas DataFrame
    rows = list()
    for row in comp10ks[['compname', 'year', 'words']].iterrows():
        r = row[1]
        for word in r.words:
            rows.append((r.compname, r.year, word))
    words = pd.DataFrame(rows, columns=['compname', 'year', 'word'])
    ##### Use Case 1: Total Word Counts by Doc
    # Calculate total number of words by doc
    counts_summary = words.groupby(['compname','year']).size().reset_index(name='count')
    ### Use Case 2: Word Counts for Risk/Uncertainty Language
    # Lay out the list of the words that we're looking for 
    wordslist = ['anticipate', 'believe', 'depend', 'fluctuate', 'indefinite', 'likelihood', 'possible', 'predict', 'risk', 'uncertain']
    # Subset bag of words by risk/uncertainty words
    word_risk = counts_full[counts_full['word'].isin(wordslist)]
    word_risk_wide = word_risk.groupby(['compname', 'year', 'word'])['count'].sum().unstack('word')
    word_risk_wide = word_risk_wide[['anticipate', 'believe', 'depend', 'fluctuate', 'likelihood', 'possible', 'predict', 'risk', 
            'uncertain']].fillna(value=0)
    # USe Case 3: Risk Topic Modeling
    # Tokenize and Preprocess
    doc_set = comp10ks['fullrisktext']
    token_text = list(build_text(doc_set.dropna()))
    # Create Bigrams 
    bigram = gensim.models.Phrases(token_text)
    # Process Texts
    texts_clean = process_texts(token_text)
    # Create Dictionary and Corpus 
    dictionary = Dictionary(texts_clean)
    corpus = [dictionary.doc2bow(text) for text in texts_clean]
    # Run the LDA Model with the optimal number of topics
    lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=20)
    opt = pd.DataFrame(numtopics, c_v)
    opt = opt.reset_index()
    s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
    lmtopics = lmlist[s].show_topics(formatted=False)
    ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
    # Prep for merging LDA models into broader dataframe 
    doc_lda = ldamodel[corpus]
    # Prep for Mallet's LDA Version 
    # Create function for Mallet's
    def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks):
        # Init output
        sent_topics_df = pd.DataFrame()
        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['risk_dominant_topic', 'risk_perc_contribution', 'risk_topic_keywords']
        # Add original text to the end of the output
        data = pd.Series(comp10ks.text.values.tolist())
        compname = pd.Series(comp10ks.compname.values.tolist())
        year = pd.Series(comp10ks.year.values.tolist())
        sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
        return(sent_topics_df)
    # Run Mallet's LDA Version 
    df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'fullrisktext' ,'compname', 'year']
    # Merge with Original Data Frame 
    Final_LDA = df_dominant_topic.merge(comp10ks, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
    # Drop Storage Heavy Text Column 
    Final_LDA = Final_LDA[['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'compname', 'year']]
    # Merge all together
    # Merge counts summaries and risk words
    Words_10k = counts_summary.merge(word_risk_wide, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    # Merge back with original info 
    comp10ks_2 = comp10ks[['year', 'compname', 'bus_address', 'mail_address']]
    Words_10k_2 = comp10ks_2.merge(Words_10k, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    Text_10k_analysis = Words_10k_2.merge(Final_LDA, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    del [[comp10ks, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, opt, texts_clean, s]]
    gc.collect()
    print("--- %s seconds ---" % (time.time() - start_time))
    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(Text_10k_analysis, os.path.join(args.model_dir, "Text_10k_analysis.joblib"))

def model_fn(model_dir):
    """Deserialized and return results

    Note that this should have the same name as the serialized model in the main method
    """
    Text_10k_analysis = joblib.load(os.path.join(model_dir, "Text_10k_analysis.joblib"))
    return Text_10k_analysis


Option 2: https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/mxnet_mnist_byom/mxnet_mnist.ipynb

In [ ]:
import os 
import json
import re
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
import s3fs
import time

# The method below collects the data from S3 along with a dictionary of 
# hyperparameters which only contains learning rate, and other parameters. (Figure out)
## DON'T REALLY NEED TO CODE THIS BECAUSE YOU CAN CREATE JOBS IN SAGEMAKER IF YOU WANT

def train(data_list, hyperparameters= {'learning_rate': 0.11}, num_cpus=0, num_gpus =1 , **kwargs):
    start_time = time.time()
    comp10ks = []
    for i in range(0,len(data_list)):
        key = data_list[i].split('/')[2]
        data_location = 's3://{}/{}/{}'.format(bucket, data_key, key)
        try:
            s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
            text_file = s[0].str.cat(sep=' ')
            comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
        except:
            pass  
    # Convert All to Pandas Dataframe 
    cols = ['year', 'compname', "text"]
    comp10ks = pd.DataFrame(comp10ks, columns=cols)
    comp10ks.Year = pd.to_datetime(comp10ks.year).dt.to_period('y')
    
    #Need to look into how to best structure the hyperparameters
    # Preprocesing
    comp10ks['text'] = comp10ks['text'].map(lambda x: re.sub('[,\.!?]', '', x))
    comp10ks['text'] = comp10ks.text.str.replace(r'<.*?>', '', regex= True)
    comp10ks['text'] = comp10ks.text.str.replace(r'&#\d+;', '', regex= True)
    # Convert the titles to lowercase
    comp10ks['text'] = comp10ks['text'].map(lambda x: x.lower())
    comp10ks['text'] = comp10ks.text.str.replace('\n', '')
    comp10ks['text'] = comp10ks.text.str.replace('\t', '')
    comp10ks['text'] = comp10ks.text.str.replace('&nbsp;','')
    # Pull out fields
    comp10ks['bus_address'] = comp10ks.text.str.extract('business address: street 1:(.*?)mail address')
    comp10ks['mail_address'] = comp10ks.text.str.extract('mail address: street 1:(.*?)10-k')
    # Scrape out risk text using url
    comp10ks['riskfac'] = comp10ks.text.str.extract('item 1a(.*?)item 2')
    comp10ks['mandisc'] = comp10ks.text.str.extract('item 7(.*?)item 8')
    comp10ks['fullrisktext'] = comp10ks['riskfac'] + comp10ks['mandisc']
    
    # Tokenize and Preprocess
    comp10ks['words'] = comp10ks['text'].map(preprocess)
    # Create Bag of Words and convert to Pandas DataFrame
    rows = list()
    for row in comp10ks[['compname', 'year', 'words']].iterrows():
        r = row[1]
        for word in r.words:
            rows.append((r.compname, r.year, word))
    words = pd.DataFrame(rows, columns=['compname', 'year', 'word'])
    ##### Use Case 1: Total Word Counts by Doc
    # Calculate total number of words by doc
    counts_summary = words.groupby(['compname','year']).size().reset_index(name='count')
    # Total Document Word Counts 
    counts_full = words.groupby(['compname','year', 'word']).size().reset_index(name='count')
    ### Use Case 2: Word Counts for Risk/Uncertainty Language
    # Lay out the list of the words that we're looking for 
    wordslist = ['anticipate', 'believe', 'depend', 'fluctuate', 'indefinite', 'likelihood', 'possible', 'predict', 'risk', 'uncertain']
    # Subset bag of words by risk/uncertainty words
    word_risk = counts_full[counts_full['word'].isin(wordslist)]
    word_risk_wide = word_risk.groupby(['compname', 'year', 'word'])['count'].sum().unstack('word')
    word_risk_wide = word_risk_wide[['anticipate', 'believe', 'depend', 'fluctuate', 'likelihood', 'possible', 'predict', 'risk', 
            'uncertain']].fillna(value=0)
    # Lemmatize - Come back and fix
    # comp10kbizdes['Text'] = map(lemma.lemmatize, comp10kbizdes['Text']
    # Tokenize and Preprocess
    doc_set = comp10ks['fullrisktext']
    token_text = list(build_text(doc_set.dropna()))
    # Create Bigrams 
    bigram = gensim.models.Phrases(token_text)
    # Process Texts
    texts_clean = process_texts(token_text)
    # Create Dictionary and Corpus 
    dictionary = Dictionary(texts_clean)
    #dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) # 15 might be prob with only 116
    corpus = [dictionary.doc2bow(text) for text in texts_clean]
    # Run the LDA Model with the optimal number of topics
    lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
    opt = pd.DataFrame(numtopics, c_v)
    opt = opt.reset_index()
    s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
    lmtopics = lmlist[s].show_topics(formatted=False)
    ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
    # Prep for merging LDA models into broader dataframe 
    doc_lda = ldamodel[corpus]
    # Prep for Mallet's LDA Version 

    # Create function for Mallet's
    def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks):
        # Init output
        sent_topics_df = pd.DataFrame()
        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['risk_dominant_topic', 'risk_perc_contribution', 'risk_topic_keywords']
        # Add original text to the end of the output
        data = pd.Series(comp10ks.text.values.tolist())
        compname = pd.Series(comp10ks.compname.values.tolist())
        year = pd.Series(comp10ks.year.values.tolist())
        sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
        return(sent_topics_df)

    # Run Mallet's LDA Version 
    df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'fullrisktext' ,'compname', 'year']
    # Merge with Original Data Frame 
    Final_LDA = df_dominant_topic.merge(comp10ks, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
    # Drop Storage Heavy Text Column 
    Final_LDA = Final_LDA[['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'compname', 'year']]
    # Merge all together
    # Merge counts summaries and risk words
    Words_10k = counts_summary.merge(word_risk_wide, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    # Merge back with original info 
    comp10ks_2 = comp10ks[['year', 'compname', 'bus_address', 'mail_address']]
    Words_10k_2 = comp10ks_2.merge(Words_10k, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    Text_10k_analysis = Words_10k_2.merge(Final_LDA, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    # Merge counts summaries and risk words
    del [[comp10ks, counts_full, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, data, opt, texts_clean, s, comp10ks_2, Words_10k_2]]
    gc.collect()
    return Text_10k_analysis
    print("--- %s seconds ---" % (time.time() - start_time))
    

Option 3: https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/pipe_bring_your_own/pipe_bring_your_own.ipynb

In [ ]:
import os 
import json
import re
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
import s3fs
import time

start_time = time.time()

comp10ks = []
for i in range(0,len(data_list)):
    key = data_list[i].split('/')[2]
    data_location = 's3://{}/{}/{}'.format(bucket, data_key, key)
    try:
        s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
        text_file = s[0].str.cat(sep=' ')
        comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
    except:
        pass  
# Convert All to Pandas Dataframe 
cols = ['year', 'compname', "text"]
comp10ks = pd.DataFrame(comp10ks, columns=cols)
comp10ks.Year = pd.to_datetime(comp10ks.year).dt.to_period('y')

#Need to look into how to best structure the hyperparameters
# Preprocesing
comp10ks['text'] = comp10ks['text'].map(lambda x: re.sub('[,\.!?]', '', x))
comp10ks['text'] = comp10ks.text.str.replace(r'<.*?>', '', regex= True)
comp10ks['text'] = comp10ks.text.str.replace(r'&#\d+;', '', regex= True)
# Convert the titles to lowercase
comp10ks['text'] = comp10ks['text'].map(lambda x: x.lower())
comp10ks['text'] = comp10ks.text.str.replace('\n', '')
comp10ks['text'] = comp10ks.text.str.replace('\t', '')
comp10ks['text'] = comp10ks.text.str.replace('&nbsp;','')
# Pull out fields
comp10ks['bus_address'] = comp10ks.text.str.extract('business address: street 1:(.*?)mail address')
comp10ks['mail_address'] = comp10ks.text.str.extract('mail address: street 1:(.*?)10-k')
# Scrape out risk text using url
comp10ks['riskfac'] = comp10ks.text.str.extract('item 1a(.*?)item 2')
comp10ks['mandisc'] = comp10ks.text.str.extract('item 7(.*?)item 8')
comp10ks['fullrisktext'] = comp10ks['riskfac'] + comp10ks['mandisc']

# Tokenize and Preprocess
comp10ks['words'] = comp10ks['text'].map(preprocess)
# Create Bag of Words and convert to Pandas DataFrame
rows = list()
for row in comp10ks[['compname', 'year', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.compname, r.year, word))
words = pd.DataFrame(rows, columns=['compname', 'year', 'word'])
##### Use Case 1: Total Word Counts by Doc
# Calculate total number of words by doc
counts_summary = words.groupby(['compname','year']).size().reset_index(name='count')
# Total Document Word Counts 
counts_full = words.groupby(['compname','year', 'word']).size().reset_index(name='count')
### Use Case 2: Word Counts for Risk/Uncertainty Language
# Lay out the list of the words that we're looking for 
wordslist = ['anticipate', 'believe', 'depend', 'fluctuate', 'indefinite', 'likelihood', 'possible', 'predict', 'risk', 'uncertain']
# Subset bag of words by risk/uncertainty words
word_risk = counts_full[counts_full['word'].isin(wordslist)]
word_risk_wide = word_risk.groupby(['compname', 'year', 'word'])['count'].sum().unstack('word')
word_risk_wide = word_risk_wide[['anticipate', 'believe', 'depend', 'fluctuate', 'likelihood', 'possible', 'predict', 'risk', 
        'uncertain']].fillna(value=0)
# Lemmatize - Come back and fix
# comp10kbizdes['Text'] = map(lemma.lemmatize, comp10kbizdes['Text']
# Tokenize and Preprocess
doc_set = comp10ks['fullrisktext']
token_text = list(build_text(doc_set.dropna()))
# Create Bigrams 
bigram = gensim.models.Phrases(token_text)
# Process Texts
texts_clean = process_texts(token_text)
# Create Dictionary and Corpus 
dictionary = Dictionary(texts_clean)
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) # 15 might be prob with only 116
corpus = [dictionary.doc2bow(text) for text in texts_clean]
# Run the LDA Model with the optimal number of topics
lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
opt = pd.DataFrame(numtopics, c_v)
opt = opt.reset_index()
s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
lmtopics = lmlist[s].show_topics(formatted=False)
ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
# Prep for merging LDA models into broader dataframe 
doc_lda = ldamodel[corpus]
# Prep for Mallet's LDA Version 

# Create function for Mallet's
def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks):
    # Init output
    sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['risk_dominant_topic', 'risk_perc_contribution', 'risk_topic_keywords']
    # Add original text to the end of the output
    data = pd.Series(comp10ks.text.values.tolist())
    compname = pd.Series(comp10ks.compname.values.tolist())
    year = pd.Series(comp10ks.year.values.tolist())
    sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
    return(sent_topics_df)

# Run Mallet's LDA Version 
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10ks)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'fullrisktext' ,'compname', 'year']
# Merge with Original Data Frame 
Final_LDA = df_dominant_topic.merge(comp10ks, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
# Drop Storage Heavy Text Column 
Final_LDA = Final_LDA[['risk_document_no', 'risk_dominant_topic', 'risk_topic_perc_contrib', 'risk_keywords', 'compname', 'year']]
# Merge all together
# Merge counts summaries and risk words
Words_10k = counts_summary.merge(word_risk_wide, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                          how='right')
# Merge back with original info 
comp10ks_2 = comp10ks[['year', 'compname', 'bus_address', 'mail_address']]
Words_10k_2 = comp10ks_2.merge(Words_10k, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                          how='right')
Text_10k_analysis = Words_10k_2.merge(Final_LDA, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                          how='right')
# Merge counts summaries and risk words
del [[comp10ks, counts_full, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, data, opt, texts_clean, s, comp10ks_2, Words_10k_2]]
gc.collect()
print("--- %s seconds ---" % (time.time() - start_time))

#### Business Descriptions

Option 1: https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/scikit_learn_iris.py

In [ ]:
import os 
import re
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
import s3fs
import time

if __name__ == '__main__':
    #### Parse Arguments
    parser = argparse.ArgumentParser()
    
    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    args = parser.parse_args()
    
    ###### All operations start here
    start_time = time.time()
    #### Bring in the data
    data_list = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    comp10ks = []
    for i in range(0,len(data_list)):
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    else: 
        try:
        s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
        text_file = s[0].str.cat(sep=' ')
        comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
        except:
            pass  
# Convert All to Pandas Dataframe 
cols = ['year', 'compname', "text"]
comp10kbizdes = pd.DataFrame(comp10kbizdes, columns=cols)
comp10kbizdes.year = pd.to_datetime(comp10kbizdes.year).dt.to_period('y')
# Preprocesing
comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: re.sub('[,\.!?]', '', x))
comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'<.*?>', '', regex= True)
comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'&#\d+;', '', regex= True)
# Convert the titles to lowercase
comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: x.lower())
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\n', '')
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\t', '')
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('&nbsp;','')
# Lemmatize - Come back and fix
# comp10kbizdes['Text'] = map(lemma.lemmatize, comp10kbizdes['Text']
# Tokenize and Preprocess
doc_set = comp10kbizdes['text']
token_text = list(build_text(doc_set.dropna()))
# Create Bigrams 
bigram = gensim.models.Phrases(token_text)
# Process Texts
texts_clean = process_texts(token_text)
# Create Dictionary and Corpus 
dictionary = Dictionary(texts_clean)
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) # 15 might be prob with only 116
corpus = [dictionary.doc2bow(text) for text in texts_clean]
# Run the LDA Model with the optimal number of topics
lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
opt = pd.DataFrame(numtopics, c_v)
opt = opt.reset_index()
s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
lmtopics = lmlist[s].show_topics(formatted=False)
ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
# Prep for merging LDA models into broader dataframe 
doc_lda = ldamodel[corpus]
# Prep for Mallet's LDA Version 

# Create function for Mallet's
def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes):
    # Init output
    sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['bizdes_dominant_topic', 'bizdes_perc_contribution', 'bizdes_topic_keywords']
    # Add original text to the end of the output
    data = pd.Series(comp10kbizdes.text.values.tolist())
    compname = pd.Series(comp10kbizdes.compname.values.tolist())
    year = pd.Series(comp10kbizdes.year.values.tolist())
    sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
    return(sent_topics_df)

# Run Mallet's LDA Version 
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords' ,'compname', 'year']
# Merge with Original Data Frame 
Final_LDA = df_dominant_topic.merge(comp10kbizdes, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
# Drop Storage Heavy Text Column 
Final_LDA_bt = Final_LDA[['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords', 'compname', 'year']]

Text_bizdes_analysis = Text_10k_analysis.merge(Final_LDA_bt, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                          how='right')
del [[comp10kbizdes, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, opt, texts_clean, s, Final_LDA_bt]]
gc.collect()
print("--- %s seconds ---" % (time.time() - start_time))
# Print the coefficients of the trained classifier, and save the coefficients
joblib.dump(Text_bizdes_analysis, os.path.join(args.model_dir, "Text_bizdes_analysis.joblib"))

def model_fn(model_dir):
    """Deserialized and return results
    
    Note that this should have the same name as the serialized model in the main method
    """
    Text_bizdes_analysis = joblib.load(os.path.join(model_dir, "Text_bizdes_analysis.joblib"))
    return Text_bizdes_analysis

Option 2

In [ ]:
import os 
import json
import re
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
import s3fs
import time

# The method below collects the data from S3 along with a dictionary of 
# hyperparameters which only contains learning rate, and other parameters. (Figure out)

def train(data_list, hyperparameters= {'learning_rate': 0.11}, num_cpus=0, num_gpus =1 , **kwargs):
    start_time = time.time()
    comp10ks = []
    for i in range(0,len(data_list)):
        key = data_list[i].split('/')[2]
        data_location = 's3://{}/{}/{}'.format(bucket, data_key, key)
        try:
            s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
            text_file = s[0].str.cat(sep=' ')
            comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
        except:
            pass  
        # Convert All to Pandas Dataframe 
    cols = ['year', 'compname', "text"]
    comp10kbizdes = pd.DataFrame(comp10kbizdes, columns=cols)
    comp10kbizdes.year = pd.to_datetime(comp10kbizdes.year).dt.to_period('y')
    # Preprocesing
    comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: re.sub('[,\.!?]', '', x))
    comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'<.*?>', '', regex= True)
    comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'&#\d+;', '', regex= True)
    # Convert the titles to lowercase
    comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: x.lower())
    comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\n', '')
    comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\t', '')
    comp10kbizdes['text'] = comp10kbizdes.text.str.replace('&nbsp;','')
    # Lemmatize - Come back and fix
    # comp10kbizdes['Text'] = map(lemma.lemmatize, comp10kbizdes['Text']
    # Tokenize and Preprocess
    doc_set = comp10kbizdes['text']
    token_text = list(build_text(doc_set.dropna()))
    # Create Bigrams 
    bigram = gensim.models.Phrases(token_text)
    # Process Texts
    texts_clean = process_texts(token_text)
    # Create Dictionary and Corpus 
    dictionary = Dictionary(texts_clean)
    #dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) # 15 might be prob with only 116
    corpus = [dictionary.doc2bow(text) for text in texts_clean]
    # Run the LDA Model with the optimal number of topics
    lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
    opt = pd.DataFrame(numtopics, c_v)
    opt = opt.reset_index()
    s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
    lmtopics = lmlist[s].show_topics(formatted=False)
    ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
    # Prep for merging LDA models into broader dataframe 
    doc_lda = ldamodel[corpus]
    # Prep for Mallet's LDA Version 

    # Create function for Mallet's
    def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes):
        # Init output
        sent_topics_df = pd.DataFrame()
        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['bizdes_dominant_topic', 'bizdes_perc_contribution', 'bizdes_topic_keywords']
        # Add original text to the end of the output
        data = pd.Series(comp10kbizdes.text.values.tolist())
        compname = pd.Series(comp10kbizdes.compname.values.tolist())
        year = pd.Series(comp10kbizdes.year.values.tolist())
        sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
        return(sent_topics_df)

    # Run Mallet's LDA Version 
    df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords' ,'compname', 'year']
    # Merge with Original Data Frame 
    Final_LDA = df_dominant_topic.merge(comp10kbizdes, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
    # Drop Storage Heavy Text Column 
    Final_LDA_bt = Final_LDA[['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords', 'compname', 'year']]

    Text_bizdes_analysis = Text_10k_analysis.merge(Final_LDA_bt, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                              how='right')
    del [[comp10kbizdes, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, opt, texts_clean, s, Final_LDA_bt]]
    gc.collect()
    return Text_bizdes_analysis
    print("--- %s seconds ---" % (time.time() - start_time))

Option 3

In [ ]:
#Import without parsing first
import sagemaker
import boto3
from sagemaker import get_execution_role
import s3fs

comp10kbizdes = []

for i in range(0,len(data_list)):
    key = data_list[i].split('/')[2]
    data_location = 's3://{}/{}/{}'.format(bucket, data_key, key)
    try:
        s = pd.read_csv(data_location, delimiter = None, error_bad_lines = False, header = None, warn_bad_lines=False)
        text_file = s[0].str.cat(sep=' ')
        comp10ks.append([data_list[i].split('/')[2].split('_')[0], ' '.join(data_list[i].split('/')[2].split('_')[3:]),text_file])
    except:
        pass   
# Convert All to Pandas Dataframe 
cols = ['year', 'compname', "text"]
comp10kbizdes = pd.DataFrame(comp10kbizdes, columns=cols)
comp10kbizdes.year = pd.to_datetime(comp10kbizdes.year).dt.to_period('y')
# Preprocesing
comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: re.sub('[,\.!?]', '', x))
comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'<.*?>', '', regex= True)
comp10kbizdes['text'] = comp10kbizdes.text.str.replace(r'&#\d+;', '', regex= True)
# Convert the titles to lowercase
comp10kbizdes['text'] = comp10kbizdes['text'].map(lambda x: x.lower())
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\n', '')
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('\t', '')
comp10kbizdes['text'] = comp10kbizdes.text.str.replace('&nbsp;','')
# Lemmatize - Come back and fix
# comp10kbizdes['Text'] = map(lemma.lemmatize, comp10kbizdes['Text']
# Tokenize and Preprocess
doc_set = comp10kbizdes['text']
token_text = list(build_text(doc_set.dropna()))
# Create Bigrams 
bigram = gensim.models.Phrases(token_text)
# Process Texts
texts_clean = process_texts(token_text)
# Create Dictionary and Corpus 
dictionary = Dictionary(texts_clean)
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) # 15 might be prob with only 116
corpus = [dictionary.doc2bow(text) for text in texts_clean]
# Run the LDA Model with the optimal number of topics
lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
opt = pd.DataFrame(numtopics, c_v)
opt = opt.reset_index()
s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
lmtopics = lmlist[s].show_topics(formatted=False)
ldamodel = LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)
# Prep for merging LDA models into broader dataframe 
doc_lda = ldamodel[corpus]
# Prep for Mallet's LDA Version 

# Create function for Mallet's
def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes):
    # Init output
    sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['bizdes_dominant_topic', 'bizdes_perc_contribution', 'bizdes_topic_keywords']
    # Add original text to the end of the output
    data = pd.Series(comp10kbizdes.text.values.tolist())
    compname = pd.Series(comp10kbizdes.compname.values.tolist())
    year = pd.Series(comp10kbizdes.year.values.tolist())
    sent_topics_df = pd.concat([sent_topics_df, data, compname, year], axis=1)
    return(sent_topics_df)

# Run Mallet's LDA Version 
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, data=comp10kbizdes)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords' ,'compname', 'year']
# Merge with Original Data Frame 
Final_LDA = df_dominant_topic.merge(comp10kbizdes, left_on=['compname', 'year'], right_on=['compname', 'year'], how='left')
# Drop Storage Heavy Text Column 
Final_LDA_bt = Final_LDA[['bizdes_document_no', 'bizdes_dominant_topic', 'bizdes_topic_perc_contrib', 'bizdes_keywords', 'compname', 'year']]

Words_10k_3 = Text_10k_analysis.merge(Final_LDA_bt, left_on=['compname', 'year'], right_on=['compname', 'year'], 
                          how='right')

del [[comp10kbizdes, counts_summary, words, word_risk, word_risk_wide, df_dominant_topic, opt, texts_clean, s, Final_LDA_bt]]
gc.collect()

# Merge it all together

In [ ]:
Master_10K_Analysis = wordrisk.merge(Final_LDA_bt, left_on=['CIK', 'Year'], right_on=['CIK', 'Year'], 
                                      how='right')

## Add Geo Coordinates

In [ ]:
!pip install geopy 
from geopy.geocoders import Nominatim
geolocator = Nominatim()
Master_10K_Analysis['Coordinates'] = Master_10K_Analysis['Address'].apply(geolocator.geocode)

Master_10K_Analysis['Latitude'] = Master_10K_Analysis.Coordinates.apply(lambda x: (x.latitude) if x is not None else x)
Master_10K_Analysis['Longitude'] = Master_10K_Analysis.Coordinates.apply(lambda x: (x.longitude) if x is not None else x)

In [ ]:
Master_10K_Analysis 

## Write out

In [ ]:
bucket='sec-edw-dev-landing-zone'
data_key = 'TBD'
data_location = 's3://{}/{}'.format(bucket, data_key)

# code you already have, saving the file locally to whatever directory you wish
file_name = "mydata.csv" 
Master_10K_Analysis.to_csv(file_name)
# instantiate S3 client and upload to s3
import boto3

s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'YOUR_S3_BUCKET_NAME', 'DESIRED_S3_OBJECT_NAME')

# -------------------------------------------------------------------------------------------

# Take 2

1. Saving an ML Model to an S3 Bucket Directory

In [ ]:
# This signifies that print_function is from a future build 
# It's basically the equivalent of "I guess you're not ready for this, but your kids are gonna love it". –
#will allow you to use print as a function:
#print('# of entries', len(dictionary), file=sys.stderr)
from __future__ import print_function

#The argparse module makes it easy to write user-friendly command-line interfaces.
import argparse
import os
import pandas as pd

# This is just about ML here, so we probably don't need
#from sklearn import tree
#from sklearn.externals import joblib

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    
    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    # So they are passing parameters here instead of within the function itself?
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    # We're continuing to add_arumgnets to the parser - here we're just saying where it comes from and where it 
    # should go to
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    # So we're activating here 
    args = parser.parse_args()
    
    # Take the set of files and read them all into a single pandas dataframe
    # Get all the file names from the appropriate directory locations 
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    # Ensure that a rogue 0 length file doesn't screw it up
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    # Read in a bunch of files together
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    # Bind output into a dataframe 
    train_data = pd.concat(raw_data)
    
    # labels are in the first column - Get target variable and predictors 
    train_y = train_data.ix[:,0]
    train_X = train_data.ix[:,1:]
    
    # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
    # as your training my require in the ArgumentParser above.
    #max_leaf_nodesint, default=None
    #Grow a tree with max_leaf_nodes in best-first fashion. 
    #Best nodes are defined as relative reduction in impurity. 
    #If None then unlimited number of leaf nodes.
    max_leaf_nodes = args.max_leaf_nodes
    
    # Now use scikit-learn's decision tree classifier to train the model.
    # Seems redundent, but here is how you specify the parameters inside of the ML function
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(train_X, train_y)
    
    # Print the coefficients of the trained classifier, and save the coefficients
    # joblib.dump: Persist an arbitrary Python object into one file.
    # We aee taking the results of the model and saving them to the model directory we set above. 
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))

    
    # This is just if we want to return a model 
    def model_fn(model_dir):
    """Deserialized and return fitted model
    
    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

# So get that this is a way to save a model to an S3 bucket in a traditional way. 
# Don't get what to do when preprocessing is involved or when you need it in a certain format. 
# Also don't get where --output-data-dir and --train play a role here. 

2. Dockerizing an R Function

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
# Define IAM role
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

import time
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

role = get_execution_role()
# So the below would be the 10k-full-text prefix for me
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-r-byo'

# For this example, we'll need 3 supporting code files.
# mars.R creates functions to fit and serve our model
    # The algorithm we've chosen to use is Multivariate Adaptive Regression Splines.

After building out the R file and the docker file 

In [ ]:
%%sh

# The above means we are in command-line mode

# The name of our algorithm
algorithm_name=sagemaker-rmars

#set -e # stop if anything fails

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
# We'd use us-east-1
region=$(aws configure get region)
region=${region:-us-west-2}

# Pulling out the ECR here 
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Train: Then to actually run it 

In [ ]:
# Typically the goal is to predict one of three flower species based on various measurements of the flowers' 
# attributes.

# This oversimplifies because we have one file
train_file = 'iris.csv'
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_file(train_file)

# Now, let's setup the information needed to train a Multivariate Adaptive Regression Splines (MARS) model on iris data.

# First, we'll get our region and account information so that we can point to the ECR container we just created.
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

- Specify the role to use
- Give the training job a name
- Point the algorithm to the container we created
- Specify training instance resources (in this case our algorithm is only single-threaded so stick to 1 instance)
- Point to the S3 location of our input data and the train channel expected by our algorithm
- Point to the S3 location for output
- Provide hyperparamters (keeping it simple)
- Maximum run time

In [ ]:
r_job = 'DEMO-r-byo-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Training job", r_job)

r_training_params = {
    "RoleArn": role,
    "TrainingJobName": r_job,
    "AlgorithmSpecification": {
        "TrainingImage": '{}.dkr.ecr.{}.amazonaws.com/sagemaker-rmars:latest'.format(account, region),
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)
    },
    "HyperParameters": {
        "target": "Sepal.Length",
        "degree": "2"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

## DON'T REALLY NEED TO CODE THIS BECAUSE YOU CAN CREATE JOBS IN SAGEMAKER IF YOU WANT


Now let's kick off our training job on Amazon SageMaker Training, using the parameters we just created. Because training is managed (AWS takes care of spinning up and spinning down the hardware), we don't have to wait for our job to finish to continue, but for this case, let's setup a waiter so we can monitor the status of our training.

In [ ]:
%%time

sm = boto3.client('sagemaker')
sm.create_training_job(**r_training_params)

status = sm.describe_training_job(TrainingJobName=r_job)['TrainingJobStatus']
print(status)
sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=r_job)
status = sm.describe_training_job(TrainingJobName=r_job)['TrainingJobStatus']
print("Training job ended with status: " + status)
if status == 'Failed':
    message = sm.describe_training_job(TrainingJobName=r_job)['FailureReason']
    print('Training failed with the following error: {}'.format(message))
    raise Exception('Training job failed')

Hosting the model we just trained takes three steps in Amazon SageMaker. First, we define the model we want to host, pointing the service to the model artifact our training job just wrote to S3.

In [ ]:
r_hosting_container = {
    'Image': '{}.dkr.ecr.{}.amazonaws.com/sagemaker-rmars:latest'.format(account, region),
    'ModelDataUrl': sm.describe_training_job(TrainingJobName=r_job)['ModelArtifacts']['S3ModelArtifacts']
}

# I.e. once we already have the model and want to predict
create_model_response = sm.create_model(
    ModelName=r_job,
    ExecutionRoleArn=role,
    PrimaryContainer=r_hosting_container)

print(create_model_response['ModelArn'])

Next, let's create an endpoing configuration, passing in the model we just registered. In this case, we'll only use a few c4.xlarges.

In [ ]:
r_endpoint_config = 'DEMO-r-byo-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': r_job,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Finally, we'll create the endpoints using our endpoint configuration from the last step.

In [ ]:
%%time

r_endpoint = 'DEMO-r-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=r_endpoint,
    EndpointConfigName=r_endpoint_config)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=r_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm.get_waiter('endpoint_in_service').wait(EndpointName=r_endpoint)
finally:
    resp = sm.describe_endpoint(EndpointName=r_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        raise Exception('Endpoint creation did not succeed')

Predict: To confirm our endpoints are working properly, let's try to invoke the endpoint.

In [ ]:

iris = pd.read_csv('iris.csv')

runtime = boto3.Session().client('runtime.sagemaker')

payload = iris.drop(['Sepal.Length'], axis=1).to_csv(index=False)
response = runtime.invoke_endpoint(EndpointName=r_endpoint,
                                   ContentType='text/csv',
                                   Body=payload)

result = json.loads(response['Body'].read().decode())
result


plt.scatter(iris['Sepal.Length'], np.fromstring(result[0], sep=','))
plt.show()

3. Build your own algorihm container (Does not show you the script of what goes in)
<br/>
<br/>
Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument train or serve. How your container processes this argument depends on the container

In [ ]:
# The below is the optimal file structure. 
# If we want test, we would just add to the data folder 
# resourceConfig.json is a JSON-formatted file that describes the network layout used for distributed training
# Since scikit-learn doesn't support distributed training, we'll ignore it here.
    # GOOD TO KNOW!
# The channels are created based on the call to CreateTrainingJob but it's generally important that channels 
# match what the algorithm expects.
# Hosting has a very different model than training because hosting is reponding to inference requests that 
# come in via HTTP. 

# Amazon SageMaker uses two URLs in the container:
# /ping will receive GET requests from the infrastructure. 
# Your program returns 200 if the container is up and accepting requests.
# /invocations is the endpoint that receives client inference POST requests.

/opt/ml
|-- input
|   |-- config
|   |   |-- hyperparameters.json
|   |   `-- resourceConfig.json
|   `-- data
|       `-- <channel_name>
|           `-- <input data>
|-- model
|   `-- <model files>
`-- output
    `-- failure

In [ ]:
# In the container directory are all the components you need to package the sample algorithm for Amazon SageMager:

.
|-- Dockerfile
|-- build_and_push.sh
`-- decision_trees
    |-- nginx.conf
    |-- predictor.py
    |-- serve
    |-- train
    `-- wsgi.py
    
# nginx.conf is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
# predictor.py is the program that actually implements the Flask web server and the decision tree predictions for this app. 
    # You'll want to customize the actual prediction parts to your application. 
    # Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate 
    # files for implementing your custom logic.
# serve is the program started when the container is started for hosting. It simply launches the gunicorn server 
    # which runs multiple instances of the Flask app defined in predictor.py. 
    # You should be able to take this file as-is.
# train is the program that is invoked when the container is run for training. 
    # You will modify this program to implement your training algorithm.
# wsgi.py is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

# In summary, the two files you will probably want to change for your application are train and predictor.py.

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

The following shell code shows how to build the container image using docker build and push the container image to ECR using docker push. This code is also available as the shell script container/build-and-push.sh, which you can run as build-and-push.sh decision_trees_sample to build the image decision_trees_sample.

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

# In Unix and Unix-like operating systems, chmod is the command and system call which is used to change the 
# access permissions of file system objects.
# So x here I think is the user getting permission
chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

# Looks like this is fairly standard and is grabbing from places withoiut me needing to custom code

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right.

The scripts are:

- train_local.sh: Run this with the name of the image and it will run training on the local tree. For example, you can run $ ./train_local.sh sagemaker-decision-trees. It will generate a model under the /test_dir/model directory. You'll want to modify the directory test_dir/input/data/... to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file input/config/hyperparameters.json to have the hyperparameter settings that you want to test (as strings).
- serve_local.sh: Run this with the name of the image once you've trained the model and it should serve the model. For example, you can run $ ./serve_local.sh sagemaker-decision-trees. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
- predict.sh: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to text/csv. For example, you can run $ ./predict.sh payload.csv text/csv.

Once you have your container packaged, you can use it to train models and use the model for hosting or batch transforms. Let's do that with the algorithm we made above.

In [ ]:
# Set up Environment 
# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

# Create the Session
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

# Upload data to default bucket
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

# Create an estimator and fit the model
<br/>
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
<br/>
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above.
<br/><br/>
Then we use fit() on the estimator to train against the data that we uploaded above.

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

# This is for a tree obviously

## DON'T REALLY NEED TO CODE THIS BECAUSE YOU CAN CREATE JOBS IN SAGEMAKER IF YOU WANT
## But you can (SDK is code way and Training Job is Console way)

# Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

In [ ]:
# Choose some data and use it for a prediction
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)


In [ ]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

# Just creating a dataset 

# Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do 
# predictions with. The serializers take care of doing the data conversions for us. 

print(predictor.predict(test_data.values).decode('utf-8'))

4. Building a Tensorflow Container
<br/>
<br/>
When you are using a framework such as Apache MXNet or TensorFlow that has direct support in SageMaker, you can simply supply the Python code that implements your algorithm using the SDK entry points for that framework. 
<br/><br/>
Won't repeat what was already shown in the previous example

# SageMaker Python SDK Local Training
To represent our training, we use the Estimator class, which needs to be configured in five steps.

1. IAM role - our AWS execution role
2. train_instance_count - number of instances to use for training.
3. train_instance_type - type of instance to use for training. For training locally, we specify local.
4. image_name - our custom TensorFlow Docker image we created.
5. hyperparameters - hyperparameters we want to pass.
<br/><br/>
Let's start with setting up our IAM role. We make use of a helper function within the Python SDK. This function throw an exception if run outside of a SageMaker notebook instance, as it gets metadata from the notebook instance. If running outside, you must provide an IAM role with proper access stated above in Permissions.

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

#### Fit, Deploy, Predict
Now that the rest of our estimator is configured, we can call fit() with the path to our local CIFAR10 dataset prefixed with file://. This invokes our TensorFlow container with 'train' and passes in our hyperparameters and other metadata as json files in /opt/ml/input/config within the container.
<br/><br/>
After our training has succeeded, our training algorithm outputs our trained model within the /opt/ml/model directory, which is used to handle predictions.
<br/><br/>
We can then call deploy() with an instance_count and instance_type, which is 1 and local. This invokes our Tensorflow container with 'serve', which setups our container to handle prediction requests through TensorFlow Serving. What is returned is a predictor, which is used to make inferences against our trained model.

In [ ]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

In [ ]:
from sagemaker.estimator import Estimator

# So they do necessarily need a whole new file. Can set up right here like this. 
hyperparameters = {'train-steps': 100}
## DON'T REALLY NEED TO CODE THIS BECAUSE YOU CAN CREATE JOBS IN SAGEMAKER IF YOU WANT

instance_type = 'local'

# Setting up the image that you already dockerized
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='sagemaker-tf-cifar10-example:latest',
                      hyperparameters=hyperparameters)

# Grabbing the data
estimator.fit('file:///tmp/cifar-10-data')

# Used data for train, and use instance count and type for deployment of the predictor
predictor = estimator.deploy(1, instance_type)

### Making predictions using Python SDK
To make predictions, we use an image that is converted using OpenCV into a json format to send as an inference request. We need to install OpenCV to deserialize the image that is used to make predictions.

In [ ]:
# ! pip install opencv-python
import cv2
import numpy

# Serialization is the process of translating data structures or object state into a format that can be 
# stored (for example, in a file or memory buffer) or transmitted and reconstructed later.
from sagemaker.predictor import json_serializer, json_deserializer

# Test data
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))

# put into right format for analysis
data = {'instances': numpy.asarray(image).astype(float).tolist()}

# The request and response format is JSON for TensorFlow Serving.
# For more information: https://www.tensorflow.org/serving/api_rest#predict_api
# Note we made this predictor previosly
predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)

# Metrics that your algorithm sends to Amazon CloudWatch during training jobs. (Keep in mind that we can use this 
# for the comparison we need from SEC

# Once it's containerized, you can also use the console or the Python SDK
# For Hyperparameter specification, specify the hyperparameters that your algorithm supports by editing the 
# JSON object. For each hyperparameter that your algorithm supports, construct a JSON block. 
    # These are just the parameters you would usually specify 

5. Create a Model Package Resource

In [ ]:
# Model artifacts can either be packaged in the same Docker container as the inference code or stored in Amazon S3.
# Validation profiles, which are batch transform jobs that Amazon SageMaker runs to test your model package's inference code.

6. Use Amazon SageMaker Containers to Run a Python Script

This sample uses the kernel: conda_tensorflow_p36

In [ ]:
# In the editor of your choice, create the following Dockerfile text file locally and save it with the file name 
# "Dockerfile" without an extension. 

# Note the freeze thing Jeremy was talking about might make this easier because I won't 
# have to load in the package individually like this

# Add and install Python modules
ADD requirements.txt "~/OneDrive - Accenture Federal Services/Documents/SEC/requirements.txt"
RUN cd /src; pip install -r requirements.txt

RUN pip install sagemaker-containers # Amazon SageMaker Containers contains the common functionality 
                                     #necessary to create a container compatible with Amazon SageMaker.

# Copies the training code inside the container
COPY train.py /opt/ml/code/train.py # copies the script to the location inside the container that is 
                                    # expected by Amazon SageMaker. The script must be located in this folder.

# Defines train.py as script entrypoint
ENV SAGEMAKER_PROGRAM train.py # defines train.py as the name of the entrypoint script that is located in 
                               #the /opt/ml/code folder for the container

Writing the actual script!

In [ ]:
# Import Packages
import os
import re
import string
import operator
import warnings
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.utils import lemmatize
from gensim.models import Phrases 
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.models import TfidfModel
from gensim.test.utils import datapath
from pprint import pprint

nltk.download('stopwords')
nltk.download('wordnet')

import gc

# Define Functions 
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(token)
    return result

def build_text(doc_set):
    for line in doc_set:
        yield gensim.utils.simple_preprocess(line, deacc=True, min_len=3)
        
def process_texts(texts):
    """
    Function to process texts. Following are the steps we take:
    
    1. Stopword Removal.
    2. Collocation detection.
    3. Lemmatization (not stem since stemming can reduce the interpretability).
    
    Parameters:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    lemmatizer = WordNetLemmatizer()
    texts = [[lemmatizer.lemmatize(word) for word in line] for line in texts]
    return texts

def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    numtopics = []
    for num_topics in range(1, limit):
        numtopics.append(num_topics)
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, per_word_topics = True, random_state=100)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
    
    return lm_list, c_v, numtopics

# The actual function 
# Convert All to Pandas Dataframe 
cols = ['year', 'compname', "text"]
comp10ks = pd.DataFrame(data, columns=cols)
comp10ks.Year = pd.to_datetime(comp10ks.year).dt.to_period('y')
# Preprocesing
comp10ks['text'] = comp10ks['text'].map(lambda x: re.sub('[,\.!?]', '', x))
comp10ks['text'] = comp10ks.text.str.replace(r'<.*?>', '', regex= True)
comp10ks['text'] = comp10ks.text.str.replace(r'&#\d+;', '', regex= True)
# Convert the titles to lowercase
comp10ks['text'] = comp10ks['text'].map(lambda x: x.lower())
comp10ks['text'] = comp10ks.text.str.replace('\n', '')
comp10ks['text'] = comp10ks.text.str.replace('\t', '')
comp10ks['text'] = comp10ks.text.str.replace('&nbsp;','')
# Pull out fields
comp10ks['bus_address'] = comp10ks.text.str.extract('business address: street 1:(.*?)mail address')
comp10ks['mail_address'] = comp10ks.text.str.extract('mail address: street 1:(.*?)10-k')
# Scrape out risk text using url
comp10ks['riskfac'] = comp10ks.text.str.extract('item 1a(.*?)item 2')
comp10ks['mandisc'] = comp10ks.text.str.extract('item 7(.*?)item 8')
comp10ks['fullrisktext'] = comp10ks['riskfac'] + comp10ks['mandisc']
# Tokenize and Preprocess
doc_set = comp10ks['fullrisktext']
token_text = list(build_text(doc_set.dropna()))
# Create Bigrams 
bigram = gensim.models.Phrases(token_text)
# Process Texts
texts_clean = process_texts(token_text)
# Create Dictionary and Corpus 
dictionary = Dictionary(texts_clean)
corpus = [dictionary.doc2bow(text) for text in texts_clean]
# Run the LDA Model with the optimal number of topics
lmlist, c_v, numtopics = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=texts_clean, limit=10)
opt = pd.DataFrame(numtopics, c_v)
opt = opt.reset_index()
s = opt[opt['index'] == opt['index'].max()].iloc[0,1]
lmtopics = lmlist[s].show_topics(formatted=False)
LdaModel(corpus=corpus, num_topics=s, id2word=dictionary, per_word_topics = True, random_state=100)

ENDED UP USING THE SCRIPT ABOVE TO FULLY DOCKERIZE. 

7. Using AWS LDA with Business Text Descriptions

Input/Output Interface for the LDA Algorithm
<br/><br/>
- LDA expects data to be provided on the train channel, and optionally supports a test channel, which is scored by the final model. LDA supports both recordIO-wrapped-protobuf (dense and sparse) and CSV file formats
- For inference, text/csv, application/json, and application/x-recordio-protobuf content types are supported

Install Packages

In [ ]:
#!conda install -y scipy

%matplotlib inline

import os, re

import boto3
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3, suppress=True)

# some helpful utility functions are defined in the Python module
# "generate_example_data" located in the same directory as this
# notebook
from generate_example_data import generate_griffiths_data, plot_lda, match_estimated_topics

# accessing the SageMaker Python SDK
import sagemaker
from sagemaker.amazon.common import numpy_to_record_serializer
from sagemaker.predictor import csv_serializer, json_deserializer

In [ ]:
# Setup AWS Credentials

from sagemaker import get_execution_role

role = get_execution_role()
bucket = '<your_s3_bucket_name_here>'
prefix = 'sagemaker/DEMO-lda-introduction'

print('Training input/output will be stored in {}/{}'.format(bucket, prefix))
print('\nIAM Role: {}'.format(role))

# Setting it up - This is where we'd pass the list of files in S3, right? 

# separate the generated data into training and tests subsets
num_documents_training = int(0.9*num_documents)
num_documents_test = num_documents - num_documents_training

documents_training = documents[:num_documents_training]
documents_test = documents[num_documents_training:]

topic_mixtures_training = topic_mixtures[:num_documents_training]
topic_mixtures_test = topic_mixtures[num_documents_training:]

print('documents_training.shape = {}'.format(documents_training.shape))
print('documents_test.shape = {}'.format(documents_test.shape))

# convert documents_training to Protobuf RecordIO format
recordio_protobuf_serializer = numpy_to_record_serializer()
fbuffer = recordio_protobuf_serializer(documents_training)

# upload to S3 in bucket/prefix/train
# We obv don't need to upload from here since it is already up - but can we convert via the console 
# (Something Sam or David might be able to do)

# Do we need to pay attention to the filepath (train) here? 
fname = 'lda.data'
s3_object = os.path.join(prefix, 'train', fname)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_object).upload_fileobj(fbuffer)

s3_train_data = 's3://{}/{}'.format(bucket, s3_object)
print('Uploaded data to S3: {}'.format(s3_train_data))

sec-edw-dev-landing-zone/10k-forms-desc

Once the data is preprocessed and available in a recommended format the next step is to train our model on the data. There are number of parameters required by SageMaker LDA configuring the model and defining the computational environment in which training will take place.
<br/><br/>
First, we specify a Docker container containing the SageMaker LDA algorithm. For your convenience, a region-specific container is automatically chosen for you to minimize cross-region data communication. Information about the locations of each SageMaker algorithm is available in the documentation.

In [ ]:
# Similar to how we would bring in our own custom

from sagemaker.amazon.amazon_estimator import get_image_uri
# select the algorithm container based on this notebook's current location

region_name = boto3.Session().region_name
container = get_image_uri(region_name, 'lda')

print('Using SageMaker LDA container: {} ({})'.format(container, region_name))

Particular to a SageMaker LDA training job are the following hyperparameters:
<br/><br/>
- num_topics - The number of topics or categories in the LDA model.
    - Usually, this is not known a priori.
    -In this example, howevever, we know that the data is generated by five topics.
- feature_dim - The size of the "vocabulary", in LDA parlance.
    - In this example, this is equal 25.
- mini_batch_size - The number of input training documents.
- alpha0 - (optional) a measurement of how "mixed" are the topic-mixtures.
    - When alpha0 is small the data tends to be represented by one or few topics.
    - When alpha0 is large the data tends to be an even combination of several or many topics.
    - The default value is alpha0 = 1.0.
<br/><br/>
In addition to these LDA model hyperparameters, we provide additional parameters defining things like the EC2 instance type on which training will run, the S3 bucket containing the data, and the AWS access role. Note that,
<br/><br/>
- Recommended instance type: ml.c4
- Current limitations:
    - SageMaker LDA training can only run on a single instance.(Could be major drawback)
    - SageMaker LDA does not take advantage of GPU hardware.
    - (The Amazon AI Algorithms team is working hard to provide these capabilities in a future release!)

In [ ]:
# We might be able to start here, but we might have to convert everything to recordIO-wrapped-protobuf first
session = sagemaker.Session()

# specify general training job information
lda = sagemaker.estimator.Estimator(
    container,
    role,
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    sagemaker_session=session,
)

# set algorithm-specific hyperparameters
lda.set_hyperparameters(
    num_topics=num_topics,
    feature_dim=vocabulary_size,
    mini_batch_size=num_documents_training,
    alpha0=1.0,
)

# run the training job on input data stored in S3
lda.fit({'train': s3_train_data})